In [1]:
import os
import pandas as pd

def calculate_proactive_lifetime_extension_rate(log_path):
    df = pd.read_csv(log_path)
    df.columns = [
        "Time", "Node", "Destination", "Previous_LifeTime", "New_LifeTime",
        "Previous_NextHop", "New_NextHop", "Previous_HopCount", "New_HopCount"
    ]

    df["LifeTimeChange"] = df["Previous_LifeTime"] != df["New_LifeTime"]
    df["NextHopChange"] = df["Previous_NextHop"] != df["New_NextHop"]
    df["HopChange"] = df["Previous_HopCount"] != df["New_HopCount"]
    df["ChangeType"] = df.apply(
        lambda row: "Hop+NextHop" if row["HopChange"] and row["NextHopChange"]
        else "HopOnly" if row["HopChange"]
        else "NextHopOnly" if row["NextHopChange"]
        else "LifeTimeOnly" if row["LifeTimeChange"]
        else "NoChange",
        axis=1
    )

    lt_only_df = df[df["ChangeType"] == "LifeTimeOnly"].copy()
    lt_only_df["TimeToExpire"] = lt_only_df["Previous_LifeTime"] - lt_only_df["Time"]
    proactive = lt_only_df[lt_only_df["TimeToExpire"] <= 3.0]

    rate = len(proactive) / len(lt_only_df) * 100 if len(lt_only_df) > 0 else 0.0
    return rate

xnodes = [0, 4, 6, 7, 9, 10, 11, 17, 19, 20]
nodes = [6, 7, 17, 18, 21, 22, 24, 26, 27, 29]
base_path = "C:/Users/Choe JongHyeon/Desktop/Applied Science/시뮬레이션 결과/Lifetime 결과_V02/중밀도/중속/"

print("=== 선제 Lifetime 연장률 (3초 이내) ===")
for node_id in nodes:
    legacy_log = os.path.join(base_path, "기존", str(node_id), "aodv_routing_log.csv")
    proposed_log = os.path.join(base_path, "제안", str(node_id), "aodv_routing_log.csv")

    if not os.path.exists(legacy_log) or not os.path.exists(proposed_log):
        continue

    legacy_rate = calculate_proactive_lifetime_extension_rate(legacy_log)
    proposed_rate = calculate_proactive_lifetime_extension_rate(proposed_log)

    print(f"[Node {node_id}] Legacy: {legacy_rate:.2f}% | Proposed: {proposed_rate:.2f}%")


=== 선제 Lifetime 연장률 (3초 이내) ===
[Node 6] Legacy: 99.81% | Proposed: 99.53%
[Node 7] Legacy: 99.99% | Proposed: 99.99%
[Node 17] Legacy: 99.42% | Proposed: 94.91%
[Node 18] Legacy: 99.71% | Proposed: 99.28%
[Node 21] Legacy: 99.09% | Proposed: 95.72%
[Node 22] Legacy: 99.59% | Proposed: 99.44%
[Node 24] Legacy: 98.94% | Proposed: 96.10%
[Node 26] Legacy: 99.65% | Proposed: 96.30%
[Node 27] Legacy: 98.23% | Proposed: 91.12%
[Node 29] Legacy: 99.27% | Proposed: 98.46%
